# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'



## Keygeneration

In [2]:
ctx = Context(EncryptionParams(128, CoeffModulus().create(128, [30]), 128 ))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

[1073738753]


In [4]:
# print(len(sk.data))
# print('secret key values : ', sk.data)

In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,1000)
ri2 = random.randint(0,1000)
ri3 = random.randint(0,1000)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[1, 0, 0, 1, 1, 1, 0, 0, 1, 1]     [1, 1, 0, 1, 0, 0, 1, 0, 1]     [1, 1, 1, 0, 1, 1, 0, 1, 1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

825
331
439


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

825     331     439


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

1156


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

1156


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

1156


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2